In [ ]:
import cv2
import numpy as np
import random
import math
import torch
from rasterio.windows import Window
from scipy.ndimage import rotate as sp_rotate

from skimage import io, draw
import torchvision
import matplotlib.pyplot as plt
import dataclasses
import os
import rasterio
from rasterio.mask import mask
import glob
import json
import gc
from torch.utils.data import Dataset
# https://www.kaggle.com/iafoss/256x256-images
mean = np.array([0.65459856,0.48386562,0.69428385])
std = np.array([0.15167958,0.23584107,0.13146145])


def crop_center(img,cropx,cropy):
    y,x,z = img.shape
    startx = x//2-(cropx//2)
    starty = y//2-(cropy//2)    
    return img[starty:starty+cropy,startx:startx+cropx]

        
class HuBMAPDatasetPreprocessing(Dataset):
    def __init__(self, image, category='train', use_compressed=False, apply_jitter=False):
        self.mask_folder = '/kaggle/input/hubmap-kidney-segmentation'
        if use_compressed:
            self.folder = '/kaggle/input/compressedhubmap'
        else:
            self.folder = '/kaggle/input/hubmap-kidney-segmentation'
        self.use_compressed = use_compressed
        self.image = image
        self.category = category
        self._file_ids = []
        self.reduce = 4 if not use_compressed else 1
        self._image = None
        self.apply_jitter = apply_jitter
        self._get_image()
        if self._image.count != 3:
            subdatasets = self._image.subdatasets
            self.layers = []
            if len(subdatasets) > 0:
                for i, subdataset in enumerate(subdatasets, 0):
                    self.layers.append(rasterio.open(subdataset))
    def _read_geometry(self):
        coords = [f['geometry'] for f in self._get_masks()]
        if self.use_compressed:
            for geom in coords:
                for coord in geom['coordinates'][0]:
                    coord[0] = coord[0] // 4
                    coord[1] = coord[1] // 4
                
        return coords
    def _get_image(self):
        if self._image:
            return self._image
        if self.use_compressed:
            file_path = os.path.join(self.folder, self.category, f'compressed-{self.image}.tiff')
        else:
            file_path = os.path.join(self.folder, self.category, f'{self.image}.tiff')

        self._image = rasterio.open(file_path)
        self.shape = self._image.shape
        self.sz = 256 * self.reduce
        self.pad0 = (self.sz - self.shape[0]%self.sz)
        self.pad1 = (self.sz - self.shape[1]%self.sz)
        self.n0max = (self.shape[0] + self.pad0)//self.sz
        self.n1max = (self.shape[1] + self.pad1)//self.sz
        if self.category == 'train':
            self.mask = rasterio.mask.mask(self._image, self._read_geometry())[0] > 0
        return self._image
    
    def close(self):
        self._get_image().close()
    
    def _get_masks(self):
        file_path = os.path.join(self.mask_folder, self.category, f'{self.image}.json')
        return json.load(open(file_path))

    def __del__(self):
        self.close()
        del self._image

    def __len__(self):
        return self.n0max * self.n1max

    def __getitem__(self, idx):
        image = self._get_image()
        n0,n1 = idx//self.n1max, idx%self.n1max
        x0,y0 = -self.pad0//2 + n0*self.sz, -self.pad1//2 + n1*self.sz
        p00,p01 = max(0,x0), min(x0+self.sz,self.shape[0])
        p10,p11 = max(0,y0), min(y0+self.sz,self.shape[1])

        img = np.zeros((self.sz,self.sz,3),np.uint8)
        if image.count == 3:
            img[(p00-x0):(p01-x0),(p10-y0):(p11-y0)] = np.moveaxis(image.read([1,2,3],
                window=Window.from_slices((p00,p01),(p10,p11))), 0, -1)
        else:
            for i,layer in enumerate(self.layers):
                img[(p00-x0):(p01-x0),(p10-y0):(p11-y0),i] =\
                  layer.read(1,window=Window.from_slices((p00,p01),(p10,p11)))

        if self.reduce != 1:
            img = cv2.resize(img,(self.sz//self.reduce,self.sz//self.reduce),
                             interpolation = cv2.INTER_AREA)
        if self.category == 'train':
            mask = np.zeros((3, self.sz,self.sz),np.uint8)

            mask[:,(p00-x0):(p01-x0),(p10-y0):(p11-y0)] = self.mask[:, p00: p01, p10: p11]
            mask = np.moveaxis(mask, 0, -1)
            if self.reduce != 1:
                mask = cv2.resize(mask, (self.sz//self.reduce,self.sz//self.reduce),
                                 interpolation = cv2.INTER_AREA)
        else:
            mask = torch.zeros(1)

        hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
        h,s,v = cv2.split(hsv)
        result_tensor = torch.from_numpy(img).permute(2,0,1).float()
        #result_tensor = torch.from_numpy((img/255.0 - mean)/std).float()
        result_tensor = torchvision.transforms.functional.normalize(result_tensor / 255, mean, std)
        result_tensor = result_tensor.float()
        if self.category == 'train' and self.apply_jitter is True:
            #jitter = torchvision.transforms.ColorJitter(brightness=0.05)
            p0 = random.random()
            p1 = random.random()
            p2 = random.random()
            if p0 <= 0.7:
                if p1 <= 0.3:
                    result_tensor = torchvision.transforms.functional.hflip(result_tensor)
                    mask = np.fliplr(mask)
                if p2 <= 0.3:
                    result_tensor = torchvision.transforms.functional.vflip(result_tensor)
                    mask = np.flipud(mask)
            else:
                angle = random.randint(-20, 20)
                result_tensor = torchvision.transforms.functional.rotate(result_tensor, angle, expand=True)
                result_tensor = torchvision.transforms.functional.center_crop(result_tensor, (256, 256))
                mask = sp_rotate(mask, angle)
                x,y,z = mask.shape
                crop = (x - 256) // 2
                leftover = (x - 2 * crop) - 256
                if crop != 0:
                    mask = mask[crop+leftover:-crop, crop+leftover:-crop]
            #result_tensor = jitter(result_tensor)

        mask = mask > 0
        if (s>40).sum() <= 1000 or img.sum() <= 1000:
            return result_tensor, mask, -1

        else: 
            return result_tensor, mask, idx

In [ ]:
!pip install torchinfo
from torchinfo import summary


In [ ]:
"""
dataset = HuBMAPDatasetPreprocessing('0486052bb', 'train', apply_jitter=True, use_compressed=True)

fig = plt.figure(figsize=(20, 12))
f, plots = plt.subplots(2, 10)
c = 0
for i in range(100):
    sample, mask, idx = dataset[i]
    sample = sample.permute(1,2,0)
    
    if idx == -1 or mask.sum() == 0:
        continue
    if c == 10:
        break
    plots[0][c].imshow(255. * sample.int().numpy())
    plots[0][c].set_title(f'Sample #{c}')
    mask = [np.array(m)*255 for m in mask]
    plots[1][c].imshow(mask)
    plots[1][c].set_title(f'Sample #{c}')
    c += 1
"""


In [ ]:
"""

dataset_test = HuBMAPDatasetPreprocessing('0486052bb', 'test', use_compressed=True)
fig = plt.figure(figsize=(20, 12))
f, plots = plt.subplots(2, 5)
c = 0
for i in range(100):
    sample, mask, idx = dataset_test[i]
    sample = sample.permute(1,2,0)
    if idx == -1:
        continue
    if c == 5:
        break
    print(sample)
    plots[0][c].imshow(sample)
    plots[0][c].set_title(f'Sample #{c}')
    c += 1
"""

In [ ]:
import torch
from torch import nn, optim
from torch.utils import data
import torchvision
from torchvision import transforms
import torchvision.transforms.functional as F
from torchvision.models.detection.rpn import AnchorGenerator
from torchvision.models.detection import FasterRCNN
from torchvision.models.segmentation import fcn_resnet101

import json





import torch.nn.functional as F

""" Parts of the U-Net model """

import torch
import torch.nn as nn
import torch.nn.functional as F

class DoubleConv(nn.Module):
    """(convolution => [BN] => ReLU) * 2"""

    def __init__(self, in_channels, out_channels, mid_channels=None):
        super().__init__()
        if not mid_channels:
            mid_channels = out_channels
        self.double_conv = nn.Sequential(
            nn.Conv2d(in_channels, mid_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(mid_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(mid_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        return self.double_conv(x)


class Down(nn.Module):
    """Downscaling with maxpool then double conv"""

    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.maxpool_conv = nn.Sequential(
            nn.MaxPool2d(2),
            DoubleConv(in_channels, out_channels)
        )

    def forward(self, x):
        return self.maxpool_conv(x)


class Up(nn.Module):
    """Upscaling then double conv"""

    def __init__(self, in_channels, out_channels, bilinear=True):
        super().__init__()

        # if bilinear, use the normal convolutions to reduce the number of channels
        #self.skip_conf = nn.Sequential(
        #    nn.Conv2d(in_channels//2,in_channels//2, kernel_size=3, padding=1),
        #    nn.BatchNorm2d(in_channels//2),
        #    nn.ReLU(inplace=True),
        #)

        if bilinear:
            self.up = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
            self.conv = DoubleConv(in_channels, out_channels, in_channels // 2)
        else:
            self.up = nn.ConvTranspose2d(in_channels , in_channels // 2, kernel_size=2, stride=2)
            self.conv = DoubleConv(in_channels, out_channels)


    def forward(self, x1, x2):
        #x2 = self.skip_conf(x2)
        
        x1 = self.up(x1)
        # input is CHW
        diffY = x2.size()[2] - x1.size()[2]
        diffX = x2.size()[3] - x1.size()[3]

        x1 = F.pad(x1, [diffX // 2, diffX - diffX // 2,
                        diffY // 2, diffY - diffY // 2])
        x = torch.cat([x2, x1], dim=1)

        return self.conv(x)


class OutConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(OutConv, self).__init__()

        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=1)

    def forward(self, x):
        return self.conv(x.clone())


class Model(nn.Module):
    """The base class for the encoder-decoder architecture."""
    def __init__(self, n_channels=3, n_classes=1, bilinear=True, **kwargs):
        super(Model, self).__init__(**kwargs)

        self.n_channels = n_channels
        self.n_classes = n_classes
        self.bilinear = bilinear

        self.inc = DoubleConv(n_channels, 64)
        self.down1 = Down(64, 128)
        self.down2 = Down(128, 256)
        factor = 2 if bilinear else 1

        self.down3 = Down(256, 512)
        self.down4 = Down(512, 1024 // factor)
        self.up1 = Up(1024, 512 // factor, bilinear)
        self.up2 = Up(512, 256 // factor, bilinear)
        self.up3 = Up(256, 128 // factor, bilinear)
        self.up4 = Up(128, 64, bilinear)
        self.outc = OutConv(64, n_classes)
        
    def forward(self, X):
        x1 = self.inc(X)

        x2 = self.down1(x1)
        x3 = self.down2(x2)
        x4 = self.down3(x3)
        x5 = self.down4(x4)
        x = self.up1(x5, x4)
        x = self.up2(x, x3)
        x = self.up3(x, x2)
        x = self.up4(x, x1)
        logits = self.outc(x)
        return (logits)

In [ ]:
import glob
import os
import numpy
import gc
import torchvision
OUTPUT_PATH = '/kaggle/working/models'

def collate_fn(x):
    x = filter(lambda x: x[-1] != -1, x)
    data.dataloader.default_collate(list(x))

#valset = ['e79de561c', 'cb2d976f4']
VALSET = ['e79de561c', 'cb2d976f4']

def get_tiff_images():
    input_folder = '/kaggle/input/hubmap-kidney-segmentation/train/*.tiff'
    for images in glob.glob(input_folder):
        bname = os.path.basename(images)
        name = os.path.splitext(bname)[0]
        if name not in VALSET:
            dataset = HuBMAPDatasetPreprocessing(os.path.splitext(bname)[0], apply_jitter=False, use_compressed=True)
            yield dataset

def get_val_set():
    for image in VALSET:
        dataset = HuBMAPDatasetPreprocessing(image, use_compressed=True)
        yield dataset

    
def dice_loss(pred,target, device):
    numerator = 2 * torch.sum(pred.float() * target.float())
    denominator = torch.sum(pred.float() + target.float())
    #print(numerator, denominator)
    return numerator / denominator


import collections
class Trainer:
    def __init__(self):
        self.is_cuda_available = torch.cuda.is_available()
        if self.is_cuda_available:
            dev = "cuda:0" 
        else:  
            dev = "cpu"
        print("Device", dev)
        self.device = torch.device(dev)
        self.model = Model()
        self.model.to(self.device)
        self.epochs = 60
        self._model_path = os.path.join(OUTPUT_PATH, 'model.pkl')
        #self._original_model_path = os.path.join(OUTPUT_PATH, 'model.pkl')
        self._original_model_path =  '../input/hubmapmodel/model (15).pkl' 

        #self._original_model_path = os.path.join(OUTPUT_PATH, 'model1.pkl') 
        self.optim = optim.Adam(self.model.parameters(), lr=0.0001)
        self.eval_metrics = []
        if os.path.exists(self._original_model_path):
            if not self.is_cuda_available:
                self._data_dict = torch.load(self._original_model_path, map_location=torch.device('cpu'))
            else:
                self._data_dict = torch.load(self._original_model_path)
            self.start_positions = self._data_dict['epoch']
            #self.loss = self._data_dict['loss']
            #self.loss = FocalTverskyLoss(self.device, alpha=0.7, beta=0.3, gamma=0.75)
            self.loss = torch.nn.BCEWithLogitsLoss(pos_weight=torch.Tensor([5]).to(self.device))
            #self.loss = lovasz_hinge
            #self.optim.load_state_dict(self._data_dict['optimizer_state_dict'])
            self.model.load_state_dict(self._data_dict['model_state_dict'])
        else:
            self.start_positions = 0
            #self.loss = FocalTverskyLoss(self.device, alpha=0.7, beta=0.3, gamma=0.75)
            self.loss = torch.nn.BCEWithLogitsLoss(pos_weight=torch.Tensor([5]).to(self.device))
            #self.loss = lovasz_hinge
    
    def evaluate(self, epoch, display=False):
        val_set = get_val_set()
        numerator, denominator = 0, 0
        TP,FP,FN = 0, 0, 0
        for ds in val_set:
            print(f'started processing image {ds.image}')
            loader = data.DataLoader(ds, 9, pin_memory=True)
            
            for batch_ndx, sample in enumerate(loader):
                if not sample:
                    continue
                target, mask, idx = sample
                target = target.to(self.device)
                mask = mask.to(self.device)
                mask = (mask[:,:,:,:1] > 0).squeeze()
                res = self.model(target)
                #print(res)
                cnt = 0
                result = (res > 0).squeeze()
                for i in range(len(mask)):
                    inputs = mask[i].int()
                    targets = result[i].int()
                    tp = (inputs * targets)    
                    fp = ((1-targets) * inputs)
                    fn = (targets * (1-inputs))
                    """
                    if display is True and (tp.sum() > 0 and (fp.sum() / tp.sum() > 0.2 or fn.sum() / tp.sum() > 0.2)):
                        _, subplots = plt.subplots(1, 3, figsize=(6,12))

                        subplots[0].imshow(target[i].permute(1,2,0).cpu())
                        subplots[1].imshow(res[i].detach().cpu().squeeze())
                        
                        display_predictions(tp, fp, fn, subplots[2])
                    """
                    
                    TP += tp.sum()
                    FP += fp.sum()
                    FN += fn.sum()
                    numerator += 2 * torch.sum(mask[i].float() * result[i].float())
                    denominator += torch.sum(mask[i].float() + result[i].float())

                    
                del mask, target, result, res
                torch.cuda.empty_cache()
                gc.collect()

        print(f'epoch {epoch} evaluation', numerator/denominator)
        print(f'epoch {epoch} TP={TP}, FP={FP}, FN={FN}')
        self.eval_metrics.append(( numerator/denominator, TP, FP, FN))
        del ds
        gc.collect()

    def train(self):
        torch.autograd.set_detect_anomaly(True)

        print(f'start position {self.start_positions}')
        if not os.path.exists(OUTPUT_PATH):
            os.makedirs(OUTPUT_PATH)
        for i in range(self.start_positions, self.epochs):
            print(f'Epoch {i}')

            for dataset in get_tiff_images():
                try:
                    loader = data.DataLoader(dataset, 12, pin_memory=True)

                    for batch_ndx, sample in enumerate(loader):
                        self.optim.zero_grad()
                        if not sample:
                            continue
                        target, mask, idx = sample
                        mask = mask[:,:,:,:1] > 0

                        result = self.model.forward(target.to(self.device)).to(self.device)
                        #print(result)

                        mask = mask.to(self.device).float().permute(0,3,1,2)
                        loss_result = self.loss(result, mask)
                        loss_result.backward()
                        self.optim.step()


                        del mask, target
                        torch.cuda.empty_cache()
                        gc.collect()

                    print(f'data processed for {dataset.image}')
                finally:
                    del dataset, loader
                    torch.cuda.empty_cache()
                    gc.collect()
            self.evaluate(i)
            with open(os.path.join(OUTPUT_PATH, 'eval_metrics.csv'), 'w+') as fw:
                fw.write('\n'.join('\t'.join(map(str, metric)) for metric in self.eval_metrics))
            print(f"saving epoch {i}")
            torch.save({
                'epoch': i,
                'model_state_dict': self.model.state_dict(),
                'optimizer_state_dict': self.optim.state_dict(),
                #'loss': self.loss,
            }, os.path.join(OUTPUT_PATH, f'model_{i}.pkl'))
            torch.save({
                'epoch': i,
                'model_state_dict': self.model.state_dict(),
                'optimizer_state_dict': self.optim.state_dict(),
                #'loss': self.loss,
            }, self._model_path)

                
def display_predictions(tp, fp, fn, subplot):
    result = np.zeros(fp.shape)
    result += tp.cpu().numpy() * 255
    #result += fp.cpu().numpy() * 128
    result += fn.cpu().numpy() * 64
    subplot.imshow(result)

In [ ]:
torch.cuda.empty_cache()
trainer = Trainer()
trainer.train()
#trainer.evaluate(1, display=True)

In [ ]:
import pandas as pd
import csv
from skimage.color import rgb2grey
def collate_fn(x):
    x = filter(lambda x: x[-1] != -1, x)
    ls = list(x)
    if ls:
        return data.dataloader.default_collate(ls)
    else:
        return []

"""    

def get_test_dataset():
    input_folder = '/kaggle/input/hubmap-kidney-segmentation/train/*.tiff'
    for images in glob.glob(input_folder):
        print(images)
        bname = os.path.basename(images)
        dataset = HuBMAPDatasetPreprocessing(os.path.splitext(bname)[0], use_compressed=True)
        yield dataset
        return
"""
def get_test_dataset():
    submission_file = '../input/hubmap-kidney-segmentation/sample_submission.csv'
    with open(submission_file) as sub_file:
        reader = csv.reader(sub_file)

        for image_id, _ in reader:
            if image_id == 'id': continue
            print(f"openning {image_id}")
            dataset = HuBMAPDatasetPreprocessing(image_id, category='test')
            yield dataset



def make_submission(model):
    names, preds = [], []
    is_cuda_available = torch.cuda.is_available()
    if is_cuda_available:
        dev = "cuda:0" 
    else:  
        dev = "cpu"
    model.to(dev)
    for ds in get_test_dataset():
        print(ds, len(ds))
        loader = data.DataLoader(ds, 32, collate_fn=collate_fn)
        mask = torch.zeros(len(ds),ds.sz,ds.sz,dtype=torch.int8)

        for batch_num, data_ in enumerate(loader):
            if not data_:
                continue
            print(f'processing batch {batch_num}')
            image, _, idx = data_
            
            image = image.to(dev)
            result = torch.nn.functional.upsample(model(image), scale_factor=ds.reduce, mode="bilinear")
            result = result.squeeze()
            print(result.shape)

            for i, ndx in enumerate(idx):
                mask[ndx] = result[i] > 0
            del result
            gc.collect()
            print(f'batch {batch_num} is done')

            #if batch_num >= 20:
            #    break
        mask = mask.view(ds.n0max,ds.n1max,ds.sz,ds.sz).\
            permute(0,2,1,3).reshape(ds.n0max*ds.sz,ds.n1max*ds.sz)
        mask = mask[ds.pad0//2:-(ds.pad0-ds.pad0//2) if ds.pad0 > 0 else ds.n0max*ds.sz,
                    ds.pad1//2:-(ds.pad1-ds.pad1//2) if ds.pad1 > 0 else ds.n1max*ds.sz]
        rle = rle_encode_less_memory(mask)
        names.append(ds.image)
        preds.append(rle)
        del mask, ds
        gc.collect()
    df = pd.DataFrame({'id':names,'predicted':preds})
    df.to_csv('submission.csv',index=False)

        
def rle_encode_less_memory(img):
    '''
    img: numpy array, 1 - mask, 0 - background
    Returns run length as string formated
    This simplified method requires first and last pixel to be zero
    '''
    pixels = img.T.flatten()
    
    # This simplified method requires first and last pixel to be zero
    pixels[0] = 0
    pixels[-1] = 0
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 2
    runs[1::2] -= runs[::2]
    
    return ' '.join(str(x) for x in runs)

model = Model()
data_dict = torch.load('/kaggle/input/hubmapmodel/model (17).pkl')
model.load_state_dict(data_dict['model_state_dict'])

#make_submission(model)

In [ ]:
"""
from skimage import io
import matplotlib.pyplot as plt
image = io.imread('/kaggle/input/compressedhubmap/train/compressed-0486052bb.tiff')
"""


In [ ]:
"""
from skimage import io
import torch
import matplotlib.pyplot as plt
subimage = image[500:1500,2500:3500]
f,subplots = plt.subplots(1, 3, figsize=(15, 45))
subplots[0].imshow(subimage)
activation = {} # dictionary to store the activation of a layer
torch.cuda.empty_cache()

def create_hook(name):
    def hook(m, i, o):
        # copy the output of the given layer
        activation[name] = o

    return hook

#trainer = Trainer()
model = Model()
data_dict = torch.load('../input/hubmapmodel/model (15).pkl', map_location=torch.device('cpu'))
model.load_state_dict(data_dict['model_state_dict'])
#model#.to('cuda:0')
input = torch.Tensor(subimage).permute(2,0,1).unsqueeze(0)#.to('cuda:0')
prediction = model(input)

pred = prediction.detach().cpu().squeeze()
subplots[1].imshow(pred)
#pred_2 = trainer.model(input).detach().cpu().squeeze()
counter = 0
#print(pred_2)
subplots[2].imshow(pred > 0)
model_weights = []
conv_layers = []
# append all the conv layers and their respective weights to the list
def rec(model):
    global counter, model_weights, conv_layers
    if type(model) in [nn.Conv2d]:
        counter += 1
        model_weights.append(model.weight)
        conv_layers.append(model)
        return
    if type(model) == nn.Upsample:
        conv_layers.append(model)
    model_children = list(model.children())
    for i in range(len(model_children)):
        if type(model_children[i]) == nn.Sequential:
            for j in range(len(model_children[i])):
                rec(model_children[i][j])
        else:
            rec(model_children[i])

rec(model)
print(f"Total convolutional layers: {len(conv_layers)}")
print(f"Total layers: {len(model_weights)}")
# take a look at the conv layers and the respective weights
for weight, conv in zip(model_weights, conv_layers):
    # print(f"WEIGHT: {weight} \nSHAPE: {weight.shape}")
    print(f"CONV: {conv} ====> SHAPE: {weight.shape}")
    # visualize the first conv layer filters

#plt.figure(figsize=(20, 17))
#for i, filter in enumerate(model_weights[0]):
    #plt.subplot(8, 8, i+1) # (8, 8) because in conv0 we have 7x7 filters and total of 64 (see printed shapes)%%
    #print(filter[0, :,:])
    #plt.imshow(filter[0, :, :].detach(), cmap='gray')
    #plt.axis('off')
    #plt.savefig('filter.png')
#plt.show()    
# pass the image through all the layers
results = [conv_layers[0](input)]
try:
    for i in range(1, len(conv_layers)):
        # pass the result from the last layer to the next layer
        res = conv_layers[i](results[-1])
        print(f'calculated {i}, conv_layert: {conv_layers[i]}, result_shape: {res.shape}')
        results.append(res)
except:
    pass
# make a copy of the `results`
outputs = results
# visualize 64 features from each layer 
# (although there are more feature maps in the upper layers)
for num_layer in range(len(outputs)):
    plt.figure(figsize=(30, 30))
    layer_viz = outputs[num_layer][0, :, :, :]
    layer_viz = layer_viz.data
    print('size', layer_viz.size())
    for i, filter in enumerate(layer_viz):
        if i == 64: # we will visualize only 8x8 blocks from each layer
            break
        plt.subplot(8, 8, i + 1)
        plt.imshow(filter, cmap='gray')
        plt.axis("off")
    print(f"Saving layer {num_layer} feature maps...")
    plt.savefig(f"layer_{num_layer}.png")
    # plt.show()
    plt.close()
"""


In [ ]:
"""
import csv
import os
import pandas as pd
test_csv = '../input/hubmapmodel/submission (3).csv'
original_csv = '../input/hubmap-kidney-segmentation/train.csv'
ds_metadata = '../input/hubmap-kidney-segmentation/HuBMAP-20-dataset_information.csv'

ds_meta = pd.read_csv(ds_metadata)
test_value = pd.read_csv(test_csv)
original_value = pd.read_csv(original_csv)


def rle_decode(mask_rle, shape):
    '''
    mask_rle: run-length as string formated (start length)
    shape: (height,width) of array to return 
    Returns numpy array, 1 - mask, 0 - background

    '''
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0]*shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    return img.reshape(shape)

image_to_shape = {}
image_to_result = {}
for index, row in ds_meta.iterrows():
    image_to_shape[os.path.splitext(row['image_file'])[0]] = (row['width_pixels'], row['height_pixels'])

for index, row in original_value.iterrows():
    image_to_result[row['id']] = row['encoding']

print(image_to_shape.keys(), image_to_result.keys(), test_value)
for index, row in test_value.iterrows():
    original = rle_decode(image_to_result[row['id']], image_to_shape[row['id']])
    result = rle_decode(row['encoding'], image_to_shape[row['id']])
"""
